In [22]:
import torch
from torch.utils.data import TensorDataset, DataLoader
#from google.colab import files
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

%matplotlib inline
from matplotlib import pyplot as plt

#import torch
#import random

#!wget https://drive.google.com/drive/folders/13461-R08muyYnBQTq14z-OG8Tmy9o-5R?usp=share_link
#df = pd.read_csv('housing.csv')

#data_url = "http://lib.stat.cmu.edu/datasets/boston"
#raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None, names=name)
#features = torch.tensor(np.hstack([df.values[::2, :], df.values[1::2, :2]]))
#labels = torch.tensor(df.values[1::2, 2])
#print(df)
#from google.colab import drive
#drive.mount('/content/drive')
#path = '/content/gdrive/My Drive/Tmp/' # the absolute path to your file
#df = pd.read_csv(path + 'housing.csv')
#drive.flush_and_unmount()
#files.upload()
#df = pd.read_csv("C:/Tmp/housing.csv", sep=" ")
#df

## Загрузка данных

In [29]:
# Boston House Price dataset has 14 features and their description is given as follows:
# CRIM per capita crime rate by town
# ZN proportion of residential land zoned for lots over 25,000 sq.ft.
# INDUS proportion of non-retail business acres per town
# CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
# NOX nitric oxides concentration (parts per 10 million)
# RM average number of rooms per dwelling
# AGE proportion of owner-occupied units built prior to 1940
# DIS weighted distances to five Boston employment centres
# RAD index of accessibility to radial highways
# TAX full-value property-tax rate per dollar 10,000.
# PTRATIO pupil-teacher ratio by town
# B 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
# LSTAT % lower status of the population
# MEDV Median value of owner-occupied homes in $1000's

# Список названий колонок
columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 
          'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

# чтение данных из файла
df = pd.read_csv("housing.csv", sep='\s+', names=columns, dtype=np.float32)

# Разделение на признаки и целевую переменную
features = df.drop('MEDV', axis=1).values
labels = df['MEDV'].values
#print(features)
#print(labels)

# # Автоматическое создание графиков
# plt.figure(figsize=(15, 20))

# # Создаем подграфики для всех 13 признаков
# for i, column in enumerate(columns[:-1]):  # Исключаем целевую переменную MEDV
#     plt.subplot(5, 3, i+1)
#     plt.title(column)
#     plt.scatter(features[:, i], labels, s=1, alpha=0.5)
#     plt.tight_layout()  # Автоматическая настройка отступов

# plt.show()

In [30]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.199997,4.0900,1.0,296.0,15.300000,396.899994,4.98,24.000000
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.900002,4.9671,2.0,242.0,17.799999,396.899994,9.14,21.600000
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.099998,4.9671,2.0,242.0,17.799999,392.829987,4.03,34.700001
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.799999,6.0622,3.0,222.0,18.700001,394.630005,2.94,33.400002
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.200001,6.0622,3.0,222.0,18.700001,396.899994,5.33,36.200001


In [35]:
# df.describe()

In [40]:
# разделение на тренировочный и тестовый наборы
x_train, x_test, y_train, y_test = train_test_split(
    features, 
    labels, 
    test_size=0.2, 
    random_state=42,
    shuffle=True  # Явное указание перемешивания
) 
print('Training Shape: ',x_train.shape)
print('Testing  Shape: ',x_test.shape)

# Преобразование numpy-массивов в тензоры
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).float()

# нормализация входных данных
x_train = torch.nn.functional.normalize(x_train)
x_test = torch.nn.functional.normalize(x_test)

Training Shape:  (404, 13)
Testing  Shape:  (102, 13)


In [9]:
batch_size = 5
 
dataset = TensorDataset(x_train, y_train)
# Выборочное чтение mini-batches
data_iter = DataLoader(dataset, batch_size, shuffle=False) #shuffle=True)

# Read a batch to see how it works
i=0
for X, y in data_iter:
    i+=1
    #print(i, X, y)
    break

In [10]:
model = torch.nn.Sequential(
    torch.nn.Linear(13, 1)
)

In [11]:
print(model[0].weight.data)

tensor([[ 0.0095, -0.0596,  0.1754,  0.2674,  0.1676,  0.2577, -0.2211, -0.1448,
          0.0951, -0.0422,  0.2136, -0.0380,  0.1756]])


In [12]:
model[0].weight.data = torch.rand(13).requires_grad_(True).reshape((1, 13))
model[0].bias.data = torch.rand(1).requires_grad_(True)
print(model[0].weight.data)
print(model[0].bias.data)

tensor([[0.4516, 0.0427, 0.1324, 0.8451, 0.9911, 0.3927, 0.9082, 0.7943, 0.6349,
         0.3607, 0.3111, 0.1030, 0.0650]])
tensor([0.4837])


In [13]:
loss = torch.nn.MSELoss(reduction='mean')

In [14]:
trainer = torch.optim.SGD(model.parameters(), lr=0.1)

In [15]:
for X, y in data_iter:
    #print(X, y) 
    break

In [16]:
num_epochs = 1000
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        trainer.zero_grad()
        l = loss(model(X).reshape(-1), y)
        l.backward()
        trainer.step()  
    l = loss(model(x_train).reshape(-1), y_train)
    if epoch % 100 == 0:
        print('epoch %d, loss: %f' % (epoch, l.item()))
        #print('epoch %d, loss: %f' % (epoch, l.item()),'|\tw', model[0].weight.data, '|\tb', model[0].bias.data)

epoch 100, loss: 56.820732
epoch 200, loss: 51.078812
epoch 300, loss: 46.818089
epoch 400, loss: 43.689186
epoch 500, loss: 41.384636
epoch 600, loss: 39.674969
epoch 700, loss: 38.394745
epoch 800, loss: 37.425388
epoch 900, loss: 36.681988
epoch 1000, loss: 36.103535


Проверка данных на тренировочном наборе

In [18]:
l = loss(model(x_train).reshape(-1), y_train)
print(l)

tensor(36.1035, grad_fn=<MseLossBackward0>)


Проверка данных на тестовом наборе

In [20]:
x_test = torch.nn.functional.normalize(x_test)
l = loss(model(x_test).reshape(-1), y_test)
print(l)

tensor(27.2349, grad_fn=<MseLossBackward0>)
